In [ ]:
import json, os
import cv2
import torch
from multiprocessing import Pool, Manager
from aquabyte.data_access_utils import S3AccessUtils, RDSAccessUtils
from aquabyte.akpd import AKPD
from aquabyte.template_matching import find_matches_and_homography
from aquabyte.biomass_estimator import NormalizeCentered2D, NormalizedStabilityTransform, ToTensor, Network
from aquabyte.akpd_scorer import generate_confidence_score
from keras.models import load_model
import boto3

import numpy as np
import plotly.express as px
import time

In [ ]:
s3_access_utils = S3AccessUtils('/root/data')
rds_access_utils = RDSAccessUtils(json.load(open(os.environ['DATA_WAREHOUSE_SQL_CREDENTIALS'])))

query = """
    SELECT * FROM
    prod.biomass_computations bc
    WHERE bc.pen_id=57
    AND bc.group_id='57'
    AND bc.captured_at between '2020-01-06' and '2020-01-07'
    AND bc.akpd_score > 0.9;
"""

df = rds_access_utils.extract_from_database(query)


<h1> Function to generate weight prediction and confidence score </h1>

In [ ]:
fig = px.histogram(df[df.akpd_score > 0.9], x='estimated_weight_g', nbins=20)
fig.show()

In [ ]:
plt.figure(figsize=(20, 10))
mask = df.score > 0.8
plt.hist(df[mask].weight, bins=10)
plt.xlabel('Predicted weight (grams)')
plt.ylabel('Frequency')
plt.title('Predicted weight distribution for IMR - 11/09-11/13')
plt.grid()
plt.show()

In [ ]:
df[mask].weight.mean()

In [ ]:
def display_crops(left_image_f, right_image_f, left_keypoints, right_keypoints, side='both', overlay_keypoints=True, show_labels=False):
    assert side == 'left' or side == 'right' or side == 'both', \
        'Invalid side value: {}'.format(side)

    if side == 'left' or side == 'right':
        fig, ax = plt.subplots(figsize=(20, 10))
        image_f = left_image_f if side == 'left' else right_image_f
        keypoints = left_keypoints if side == 'left' else right_keypoints
        image = plt.imread(image_f)
        ax.imshow(image)

        if overlay_keypoints:
            for bp, kp in keypoints.items():
#                 ax.scatter([kp[0]], [kp[1]], color='red', s=1)
                ax.scatter([kp[0]], [kp[1]], color='red', s=200, alpha=0.3)
                if show_labels:
                    ax.annotate(bp, (kp[0], kp[1]), color='red')
    else:
        fig, axes = plt.subplots(2, 1, figsize=(20, 20))
        left_image = plt.imread(left_image_f)
        right_image = plt.imread(right_image_f)
        axes[0].imshow(left_image)
        axes[1].imshow(right_image)
        if overlay_keypoints:
            for bp, kp in left_keypoints.items():
#                 axes[0].scatter([kp[0]], [kp[1]], color='red', s=1)
                axes[0].scatter([kp[0]], [kp[1]], color='red', s=200, alpha=0.3)
                if show_labels:
                    axes[0].annotate(bp, (kp[0], kp[1]), color='red')
            for bp, kp in right_keypoints.items():
#                 axes[1].scatter([kp[0]], [kp[1]], color='red', s=1)
                axes[1].scatter([kp[0]], [kp[1]], color='red', s=200, alpha=0.3)
                if show_labels:
                    axes[1].annotate(bp, (kp[0], kp[1]), color='red')
    plt.show()

In [ ]:
idx = 24
row = df[mask].sort_values('weight', ascending=False).iloc[idx]

left_crop_url, right_crop_url = row.left_crop_url, row.right_crop_url
left_image_f, _, _ = s3_access_utils.download_from_url(left_crop_url)
right_image_f, _, _ = s3_access_utils.download_from_url(right_crop_url)
left_crop_metadata, right_crop_metadata = row.left_crop_metadata, row.right_crop_metadata,
cm = row.camera_metadata
row_id = idx

# run AKPD
akpd_keypoints = akpd.predict_keypoints(left_crop_url, right_crop_url, left_crop_metadata, right_crop_metadata)
left_keypoints = {item['keypointType']: [item['xCrop'], item['yCrop']] for item in akpd_keypoints[0]['leftCrop']}
right_keypoints = {item['keypointType']: [item['xCrop'], item['yCrop']] for item in akpd_keypoints[0]['rightCrop']}


# run AKPD scoring network
input_sample = {
    'keypoints': akpd_keypoints[0],
    'cm': cm,
    'stereo_pair_id': row_id,
    'single_point_inference': True
}
nomralized_centered_2D_kps = \
    normalize_centered_2D_transform_akpd.__call__(input_sample)

akpd_normalized_kps = akpd_normalization_transform.__call__(nomralized_centered_2D_kps)
tensorized_kps = to_tensor_transform.__call__(akpd_normalized_kps)
score = akpd_scorer_network(tensorized_kps['kp_input']).item()
display_crops(left_image_f, right_image_f, left_keypoints, right_keypoints, show_labels=True)


In [ ]:
left_crop_url

In [ ]:
3250 = 2070 * (1+r)**60

In [ ]:
((3250/2070)**(1/63.0)) - 1

In [ ]:
2070*(1.00718)**(51)